#### MinAbsSum solution

해당 문제를 처참히 틀리고 [official solution](https://codility.com/media/train/solution-min-abs-sum.pdf) 을 보고 이해한 내용을 정리, 변수명 등을 좀 더 이해하기 쉽게 변경했습니다.

official solution은 주어진 배열의 인자들을 모두 절대값 처리 한 다음 두 그룹으로 나눕니다. 그 두 그룹을 group_A, group_B라고 한다면, group_A의 합과 group_B의 합의 최소차이를 구하는 방법으로 바꾸어서 문제를 풉니다.

group_A의 합을 P, group_B의 합을 Q, 주어진 배열의 모든 인자들의 합을 S라고 하고, P <= Q 라고 합시다. 그렇다면 P가 0 <= P <= S / 2 범위에 있는 수 중 가장 큰 숫자이어야지 Q - P 가 최소가 될 것 입니다. (P 가 S / 2 와 가까워야 한다는 뜻입니다.) 

일단 동적계획법을 이용해서 주어진 모든 인자로 생성할 수 있는 부분합들을 모두 구합니다. 

dp[N] -> 주어진 배열의 인자로 N 을 만들 수 있는가?

In [5]:
def solution(A):
    A = [abs(A[i]) for i in range(len(A))] # O(N)
    total_sum = sum(A) # O(N)
    dp = [0] * (total_sum + 1)
    dp[0] = 1 # 0은 A의 인자를 하나도 안 쓰고 만들 수 있습니다.
    for a in A: # O(N * total_sum)
        for partial_sum in range(total_sum, -1, -1):
            if dp[partial_sum] == 0:
                continue        
            # partial_sum 을 만들 수 있다면 partial_sum 에 a 를 더해 또 다른 partial_sum을 만들 수 있습니다.
            if partial_sum + a <= total_sum:
                dp[partial_sum + a] = 1
    for partial_sum in range(total_sum // 2, -1, -1):
        if dp[partial_sum] == 1:
            return total_sum - 2 * partial_sum
    return 0

[Task Score 63% Correctness 100% Performance 20%](https://app.codility.com/demo/results/training3DETUN-PTP/)

성능 점수가 낮게 나왔습니다. 문제점은 배열의 중복되는 인자들 입니다.

문제의 입력 범위를 보면

* N is an integer within the range [0..20,000]
* each element of array A is an integer within the range [−100..100]

배열의 범위가 0~20,000 인데 각 인자들의 범위는 -100 ~ 100 밖에 되지 않습니다. 
코드에서 각 인자들을 절대값으로 처리하니까 결국 범위는 0 ~ 100이 되겠네요. 
자연스럽게 중복되는 인자들이 많을것 입니다. 

동적계획법을 좀 수정하면 중복되는 값들을 계산하지 않아도 됩니다.

dp[N] == -1 이라면 주어진 배열로 N 을 만들 수 없다.
dp[N] >= 0 이라면 주어진 배열의 인자로 N을 만들고 몇개가 남아있는가? 


In [7]:
def solution(A):
    if len(A) == 0:
        return 0
    A = [abs(A[i]) for i in range(len(A))] # O(N)
    
    max_value = max(A)    
    count = [0] * (max_value + 1)
    for a in A:
        count[a] += 1
        
    total_sum = sum(A) # O(N)
    
    dp = [-1] * (total_sum + 1)
    dp[0] = 0
    
    for a in range(1, max_value + 1): #O(max_value * total_sum)
        if count[a] <= 0:
            continue
        for partial_sum in range(total_sum):
            if dp[partial_sum] >= 0: # partial_sum을 만드는데 a는 사용하지 않았으므로 
                dp[partial_sum] = count[a] # dp[partial_sum]을 count[a] 로 초기화
                
            elif partial_sum >= a and dp[partial_sum - a] > 0: # partial_sum - a 를 만들고 남은 a의 수가 0 보다 크면
                dp[partial_sum] = dp[partial_sum - a] - 1 # partial_sum 을 만들면 그 남은 수에서 또 a를 썼으니 -1 을 해줍니다.
                
    for partial_sum in range(total_sum // 2, -1, -1):
        if dp[partial_sum] >= 0:
            return total_sum - 2 * partial_sum
    return 0

[Task Score 100% Correctness 100% Performance 100%](https://app.codility.com/demo/results/training5YPG7K-DEK/)

문제를 바꿔서 생각하는 발상이랑, 중복되는 인자들을 한번에 계산하는 테크닉이 멋진 풀이인거 같습니다.